In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
import scanpy as sc
import igraph
import scvelo as scv
import loompy as lmp
import anndata

import warnings
warnings.filterwarnings('ignore')
from scipy import io
from scipy.sparse import coo_matrix, csr_matrix
import os 

In [ ]:
# load sparse matrix:
X = io.mmread("C:\\Users\\arshe\\OneDrive\\Desktop\\Working data\\scRNAseq\\nkx3.1_52hpf_2\\IP\\counts.mtx")
# create anndata object
adata = anndata.AnnData(
    X=X.transpose().tocsr()
)
# load cell metadata:
cell_meta = pd.read_csv("C:\\Users\\arshe\\OneDrive\\Desktop\\Working data\\scRNAseq\\nkx3.1_52hpf_2\\IP\\metadata.csv")
# load gene names:
with open("C:\\Users\\arshe\\OneDrive\\Desktop\\Working data\\scRNAseq\\nkx3.1_52hpf_2\\IP\\gene_names.csv", 'r') as f:
    gene_names = f.read().splitlines()

In [ ]:
# set anndata observations and index obs by barcodes, var by gene names
adata.obs = cell_meta
adata.obs.index = adata.obs['barcode']
adata.var.index = gene_names

# load dimensional reduction:
pca = pd.read_csv("C:\\Users\\arshe\\OneDrive\\Desktop\\Working data\\scRNAseq\\nkx3.1_52hpf_2\\IP\\pca.csv")
pca.index = adata.obs.index

# set pca and umap
adata.obsm['X_pca'] = pca.to_numpy()
adata.obsm['X_umap'] = np.vstack((adata.obs['UMAP_1'].to_numpy(), adata.obs['UMAP_2'].to_numpy())).T

# plot a UMAP colored by sample to test:
sc.pl.umap(adata, color=['identity'], frameon=False, save=True)

In [ ]:

scv.set_figure_params(style='scvelo')
pl.rcParams['figure.figsize'] = (10,10)
loom = scv.read("C:\\Users\\arshe\\OneDrive\\Desktop\\Working data\\scRNAseq\\nkx3.1_52hpf_2\\raw\\nkx3_1_52hpf.loom", cache=True)
# rename barcodes in order to merge:
barcodes = [bc.split(':')[1] for bc in loom.obs.index.tolist()]
barcodes = [bc[0:len(bc)-1] + '_10' for bc in barcodes]
loom.obs.index = barcodes
loom.var_names_make_unique()
adata = scv.utils.merge(adata, loom)

In [ ]:
# plot umap to check
sc.pl.umap(adata, color='identity', frameon=False, legend_loc='on data')
scv.pp.filter_and_normalize(adata)
scv.pp.moments(adata)
scv.tl.recover_dynamics(adata)
scv.tl.velocity(adata, mode='dynamical')
graph = scv.tl.velocity_graph(adata)
scv.tl.recover_latent_time(adata)


In [ ]:
scv.pl.velocity_embedding_grid(adata, basis='umap', color='identity', title='', scale=0.25, arrow_size= 2, min_mass = 0, density =1)
scv.pl.velocity_embedding_stream(adata, basis='umap', title='', legend_loc='none',color='identity', palette=[ '#808080', '#698B69', '#FF7F50',  '#EE82EE', '#483D8B', '#00BFFF', '#CD1076'], arrow_size=3, density=3, min_mass=3)

In [ ]:
scv.tl.velocity_confidence(adata)
keys = 'velocity_length', 'velocity_confidence'
scv.pl.scatter(adata, c=keys, cmap='coolwarm', perc=[5, 95])
##length = speed/rate of differentiation

df = adata.obs.groupby('identity')[keys].mean().T
df

In [ ]:
adata.write('C:\\Users\\arshe\\OneDrive\\Desktop\\Working data\\scRNAseq\\nkx3.1_52hpf_2\\IP\\Analyzed.h5ad')